In [1]:
# Dataset from: https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=train.csv

In [2]:


import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt



In [3]:
df_train = pd.read_csv("train.csv")
df_train.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(100000, 28)

In [4]:
df_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,84998.0,4194.170850,3183.686167,303.645417,1625.568229,3093.745000,5957.448333,15204.633333
Num_Bank_Accounts,100000.0,17.091280,117.404834,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,100000.0,22.474430,129.057410,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,100000.0,72.466040,466.422621,1.000000,8.000000,13.000000,20.000000,5797.000000
Delay_from_due_date,100000.0,21.068780,14.860104,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,98035.0,27.754251,193.177339,0.000000,3.000000,6.000000,9.000000,2597.000000
Credit_Utilization_Ratio,100000.0,32.285173,5.116875,20.000000,28.052567,32.305784,36.496663,50.000000
Total_EMI_per_month,100000.0,1403.118217,8306.041270,0.000000,30.306660,69.249473,161.224249,82331.000000


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [6]:
df_train.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [7]:
df_train = df_train.dropna()

In [8]:
# get rid of columns that don't help
df_train.drop(['ID', 'Month', 'Customer_ID', 'Name', 'SSN'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
df_train.head(50)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
6,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
8,28_,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,7,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good
10,28,Teacher,34847.84_,3037.986667,2,1385,6,1,Credit-Builder Loan,3,...,_,605.03,33.224951,26 Years and 9 Months,No,18.816215,58.51597569589465,High_spent_Large_value_payments,466.46647639764313,Standard
12,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,34.977895,26 Years and 11 Months,No,18.816215,130.11542024292334,Low_spent_Small_value_payments,444.8670318506144,Good
13,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,33.381010,27 Years and 0 Months,No,18.816215,43.477190144355745,High_spent_Large_value_payments,481.505261949182,Good
15,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,32.933856,27 Years and 2 Months,No,18.816215,218.90434353388733,Low_spent_Small_value_payments,356.07810855965045,Good
16,34,_______,143162.64,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",5,...,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
20,34,_______,143162.64,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",10,...,Good,1303.01,31.376150,18 Years and 1 Months,No,246.992319,430.9475278803298,Low_spent_Large_value_payments,810.7821526659284,Good


In [10]:
## Nominal Data - Occupation, Type_of_Loan, Payment_Behaviour
## Continuous Value - Annual_Income, Monthly_Inhand_Salary, Changed_Credit_Limit, Outstanding_Debt, 
    # Credit_Utilization_Ratio, Total_EMI_per_month, Amount_invested_monthly, Monthly_Balance
## Counts - Num_Bank_Accounts, Num_Credit_Card, Num_of_Loan, Delay_from_due_date, Num_of_Delayed_Payment, Num_Credit_Inquiries
## Integer - Interest_Rate
## Ordinal - Credit_Mix, 
## Binary - Payment_of_Min_Amount
## <<to parse>> - Credit_History_Age

## <<Prediction Label - Credit_Score (ordinal)

In [11]:
def convert_to_float(feature, df):
  df = df[pd.to_numeric(df[feature], errors='coerce').notnull()]
  df[feature] = df[feature].astype(float, errors = 'raise')
  return df

In [12]:
# Convert Annual_Income, Monthly_Inhand_Salary, Changed_Credit_Limit, Outstanding_Debt, 
    # Credit_Utilization_Ratio, Total_EMI_per_month, Amount_invested_monthly, Monthly_Balance
    # to float

df_train = convert_to_float('Age', df_train)
df_train = convert_to_float('Annual_Income', df_train)
df_train = convert_to_float('Monthly_Inhand_Salary', df_train)
df_train = convert_to_float('Changed_Credit_Limit', df_train)
df_train = convert_to_float('Outstanding_Debt', df_train)
df_train = convert_to_float('Credit_Utilization_Ratio', df_train)
df_train = convert_to_float('Total_EMI_per_month', df_train)
df_train = convert_to_float('Amount_invested_monthly', df_train)
df_train = convert_to_float('Monthly_Balance', df_train)

df_train = convert_to_float('Num_of_Loan', df_train)
df_train = convert_to_float('Num_of_Delayed_Payment', df_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40147 entries, 0 to 99997
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       40147 non-null  float64
 1   Occupation                40147 non-null  object 
 2   Annual_Income             40147 non-null  float64
 3   Monthly_Inhand_Salary     40147 non-null  float64
 4   Num_Bank_Accounts         40147 non-null  int64  
 5   Num_Credit_Card           40147 non-null  int64  
 6   Interest_Rate             40147 non-null  int64  
 7   Num_of_Loan               40147 non-null  float64
 8   Type_of_Loan              40147 non-null  object 
 9   Delay_from_due_date       40147 non-null  int64  
 10  Num_of_Delayed_Payment    40147 non-null  float64
 11  Changed_Credit_Limit      40147 non-null  float64
 12  Num_Credit_Inquiries      40147 non-null  float64
 13  Credit_Mix                40147 non-null  object 
 14  Outsta

In [14]:
print(df_train['Occupation'].value_counts())

_______          2804
Lawyer           2688
Mechanic         2614
Accountant       2537
Teacher          2532
Doctor           2531
Engineer         2529
Architect        2528
Developer        2521
Scientist        2488
Media_Manager    2479
Entrepreneur     2461
Journalist       2381
Musician         2374
Manager          2351
Writer           2329
Name: Occupation, dtype: int64


In [15]:
print(df_train['Type_of_Loan'].value_counts())

Not Specified                                                                                                  664
Personal Loan                                                                                                  590
Credit-Builder Loan                                                                                            583
Student Loan                                                                                                   580
Debt Consolidation Loan                                                                                        574
                                                                                                              ... 
Home Equity Loan, Payday Loan, Debt Consolidation Loan, Student Loan, and Credit-Builder Loan                    1
Student Loan, Student Loan, Credit-Builder Loan, Home Equity Loan, Debt Consolidation Loan, and Payday Loan      1
Payday Loan, Mortgage Loan, Home Equity Loan, Payday Loan, Auto Loan, and Auto L

In [16]:
# Remove bad interest rates

df_train = df_train[df_train.Interest_Rate < 50]
print(df_train['Interest_Rate'].value_counts())

5     1851
8     1786
12    1775
6     1763
10    1741
9     1731
18    1731
15    1644
20    1625
7     1610
17    1600
11    1585
16    1555
19    1501
3     1028
13     959
1      954
14     928
4      919
2      916
30     799
24     779
29     771
23     769
32     765
22     734
28     731
21     726
27     704
34     696
25     673
31     653
26     644
33     635
Name: Interest_Rate, dtype: int64


In [17]:
print(df_train['Payment_Behaviour'].value_counts())

Low_spent_Small_value_payments      9926
High_spent_Medium_value_payments    7065
Low_spent_Medium_value_payments     5393
High_spent_Large_value_payments     5353
High_spent_Small_value_payments     4522
Low_spent_Large_value_payments      4095
!@9#%8                              2927
Name: Payment_Behaviour, dtype: int64


In [18]:
# Deal with categories

df = pd.get_dummies(df_train['Payment_Behaviour'], prefix='Payment_Behaviour')
df_train = pd.concat([df_train, df], axis=1).drop('Payment_Behaviour', axis=1)

df = pd.get_dummies(df_train['Occupation'], prefix='Occupation')
df_train = pd.concat([df_train, df], axis=1).drop('Occupation', axis=1)

df = pd.get_dummies(df_train['Type_of_Loan'], prefix='Type_of_Loan')
df_train = pd.concat([df_train, df], axis=1).drop('Type_of_Loan', axis=1)

df = pd.get_dummies(df_train['Payment_of_Min_Amount'], prefix='Payment_of_Min_Amount')
df_train = pd.concat([df_train, df], axis=1).drop('Payment_of_Min_Amount', axis=1)


In [19]:
# Deal with ordinal variables

## Ordinal - Credit_Mix, Credit_Score

In [20]:
print(df_train['Credit_Mix'].value_counts())

Standard    14388
Good         8539
Bad          8331
_            8023
Name: Credit_Mix, dtype: int64


In [21]:
Credit_Mix_mapper = {'Bad':0, 'Standard':1, 'Good':2, '_':0}
df_train['Credit_Mix'] = df_train['Credit_Mix'].replace(Credit_Mix_mapper)

In [22]:
print(df_train['Credit_Score'].value_counts())

Standard    20778
Poor        12172
Good         6331
Name: Credit_Score, dtype: int64


In [23]:
df_train.Credit_History_Age.describe()


count                     39281
unique                      404
top       18 Years and 4 Months
freq                        198
Name: Credit_History_Age, dtype: object

In [24]:


import regex as re

df_train.Credit_History_Age = df_train.Credit_History_Age.str.extract('(\d+)').astype(int)

In [25]:
df_train.Credit_History_Age.astype(str).describe()

count     39281
unique       34
top          18
freq       2068
Name: Credit_History_Age, dtype: object

In [26]:
df_train_true = df_train.pop('Credit_Score')
df_train_true = df_train_true.to_frame()

df = pd.get_dummies(df_train_true['Credit_Score'], prefix='Credit_Score')
df_train_true = pd.concat([df_train_true, df], axis=1).drop('Credit_Score', axis=1)

In [27]:
df_train_true

,Credit_Score_Good,Credit_Score_Poor,Credit_Score_Standard
0,1,0,0
9,1,0,0
12,1,0,0
13,1,0,0
15,1,0,0
...,...,...,...
99989,0,0,1
99991,0,0,1
99994,0,1,0
99995,0,1,0


In [28]:
normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=1)
normalizer.adapt(np.array(df_train.astype('float32')))

In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39281 entries, 0 to 99996
Columns: 6263 entries, Age to Payment_of_Min_Amount_Yes
dtypes: float64(12), int64(6), uint8(6245)
memory usage: 239.6 MB


In [30]:


## Callbacks

# End training if no improvement seen
early_stop_callbk = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.05, patience=5)

# Optimize LR
opt_LR_callbk = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2
)



In [31]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  return model

In [32]:


model_1 = get_basic_model()
history_1 = model_1.fit(df_train, df_train_true, epochs=15, batch_size=32, validation_split=0.2, 
                callbacks=[early_stop_callbk, opt_LR_callbk])



Epoch 1/15
982/982 [==============================] - 7s 4ms/step - loss: 4.7396 - accuracy: 0.4503 - val_loss: 4.4312 - val_accuracy: 0.4238 - lr: 0.0010
Epoch 2/15
982/982 [==============================] - 3s 3ms/step - loss: 4.3897 - accuracy: 0.4626 - val_loss: 4.4518 - val_accuracy: 0.4106 - lr: 0.0010
Epoch 3/15
982/982 [==============================] - 3s 3ms/step - loss: 4.3047 - accuracy: 0.4675 - val_loss: 4.6022 - val_accuracy: 0.4210 - lr: 0.0010
Epoch 4/15
982/982 [==============================] - 3s 3ms/step - loss: 4.3936 - accuracy: 0.4727 - val_loss: 4.5891 - val_accuracy: 0.4199 - lr: 1.0000e-04
Epoch 5/15
982/982 [==============================] - 3s 3ms/step - loss: 4.3999 - accuracy: 0.4734 - val_loss: 4.6073 - val_accuracy: 0.4190 - lr: 1.0000e-04
Epoch 6/15
982/982 [==============================] - 3s 3ms/step - loss: 4.3872 - accuracy: 0.4733 - val_loss: 4.6175 - val_accuracy: 0.4189 - lr: 1.0000e-05
